#  INF8225 Projet QA : HyDE vs RAG-Fusion

## Import des librairies

In [1]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Optional
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
import nltk
import faiss
import torch.nn.functional as F
from torch.utils.data import Dataset

import os
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
import itertools
import time
import warnings

c:\Users\mathi\anaconda3\envs\rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Déclarations des constantes

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16

## Chargement des données

In [3]:
root_path = './'
data_path = root_path + 'data/'
questions_test = pd.read_csv(data_path + 'questions_test.csv')
questions_train = pd.read_csv(data_path+'questions_train.csv')
questions_val = pd.read_csv(data_path+'questions_val.csv')
texts = pd.read_csv(data_path + 'texts.csv')

# Colonnes utiles
passages = texts['text'].tolist()
questions_t = questions_test['question'].tolist()
questions_tr = questions_train['question'].tolist()
questions_v = questions_val['question'].tolist()

## Encoding

### Chargement de l'encodeur

In [4]:
from transformers import AutoTokenizer, AutoModel
model_name = 'BAAI/bge-base-en-v1.5'
emb_tokenizer = AutoTokenizer.from_pretrained(model_name)
emb_model = AutoModel.from_pretrained(model_name)
emb_model.to(DEVICE)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
)

emb_model = get_peft_model(emb_model, lora_config)

In [6]:
emb_model.load_state_dict(torch.load("models_encoder/fine_tuned_model_weights_epoch_8.pth"))

C:\Users\mathi\AppData\Local\Temp\ipykernel_15368\1657793998.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb_model.load_state_dict(torch.load("models_encoder/fine_tu

<All keys matched successfully>

### Stockage dans FAISS

In [9]:
def encode_sequences(sequences: list, tokenizer, model, device=DEVICE, batch_size=BATCH_SIZE, training=False) -> torch.Tensor:
    model.train() if training else model.eval()
    context = torch.enable_grad() if training else torch.no_grad()

    embeddings = []
    with context:
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i:i + batch_size]

            # Tokenisation des textes avant entrée du modèle
            inputs = tokenizer(
                batch, padding=True, truncation=True, return_tensors="pt"
            ).to(device)

            # Passage dans le modèle encodeur
            output = model(**inputs).last_hidden_state

            # Pooling : moyenne des vecteurs de tous les tokens pour chaque séquence
            pooled = output.mean(dim=1)  # [batch_size, hidden_size]

            embeddings.append(pooled)

    # Concatène tous les embeddings pour obtenir un seul tensor de sortie
    return torch.cat(embeddings, dim=0)

In [8]:
passage_embed = encode_sequences(passages, emb_tokenizer, emb_model)
passage_embeddings = passage_embed.cpu().numpy()
faiss.normalize_L2(passage_embeddings) #  Normalisation des vecteurs pour l'approche similarité cosinus
d = passage_embeddings.shape[1]
index = faiss.IndexFlatIP(d)  # Index Faiss
index.add(passage_embeddings)

In [10]:
def retrieve_top_k_passages_faiss(questions: list, vector_index: faiss.IndexFlatL2, embedding_model_tokenizer, embedding_model, k: int):
    question_embeddings = encode_sequences(questions, embedding_model_tokenizer, embedding_model)
    question_embeddings = question_embeddings.cpu().numpy()
    faiss.normalize_L2(question_embeddings)
    _, top_k_indices = vector_index.search(question_embeddings, k)
    
    return top_k_indices

## Generator

In [ ]:
from huggingface_hub import login
login(token = '')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\mathi\.cache\huggingface\token
Login successful


### Chargement du Generateur

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

gen_tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",           # auto-placement sur GPU si dispo
    torch_dtype=torch.float16,   # allège la mémoire (fp16)
    trust_remote_code=True,       # pour activer les modèles qui utilisent du code perso
)
gen_tokenizer.pad_token = gen_tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

c:\Users\mathi\anaconda3\envs\ml_env\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mathi\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


### Prompts

In [22]:
def generate_prompts(questions, index, emb_tokenizer, emb_model, k) : 
    prompts = []
    faiss_passages = retrieve_top_k_passages_faiss(questions, index, emb_tokenizer, emb_model, k)  
    help_instruction = "Give a concise answer to the question."

    # Boucle sur chaque question et ses passages associés
    for i, passages_id in enumerate(faiss_passages):
        passages = []

        # Récupérer les textes des passages associés à la question actuelle
        for passage_id in passages_id:
            passage_text = texts[texts['id'] == passage_id]['text'].iloc[0]
            passages.append(passage_text)

        # Construire le contexte complet et l'ajouter à la liste des prompts
        context_string = (
            f"{help_instruction}\n" +
            "\n".join([f"{passage}" for j, passage in enumerate(passages)]) +
            "\n" +
            f"{questions[i]}\n"
        )
        prompts.append(context_string)
    return prompts

In [23]:
k_values = [1, 2, 3, 4]
prompts_k = {}

for k in k_values:
    prompts_k[k] = generate_prompts(questions_v, index, emb_tokenizer, emb_model, k)

print(prompts_k[2][0])

Give a concise answer to the question.
 Polymorphism in the CYSLTR2 gene has been associated with the inheritance of asthma in separate populations. The M201V variant of CYSLTR2 exhibits decreased responsiveness to LTD4, suggesting that this hypo-responsiveness underlies its asthma transmission-protecting effect.
 The human CYSLTR2 gene maps to chromosome 13 and consists of four exons. The protein encoded by CYSLTR2 is composed of 347 amino acids and shares only 31% amino acid identity with the CysLTR1 protein. CYSLTR2 is a G protein-coupled receptor that activates the Gq alpha subunit and/or Ga subunit of its coupled G protein when bound to its CysLT ligands.
What is the effect of the M201V variant of CYSLTR2 on responsiveness to LTD4?



### Generation

In [32]:
def process_batch_prompts(batch_prompts, tokenizer, model, device, max_new_tokens=64):
    formatted_prompts = [
        f"<s>[INST] {prompt.strip()} [/INST]" for prompt in batch_prompts
    ]

    print(formatted_prompts)

    inputs = tokenizer(formatted_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

    # Génération
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

    # Décodage et extraction de la réponse
    responses = [
        tokenizer.decode(output, skip_special_tokens=True).split('[/INST]')[-1].strip()
        for output in outputs
    ]

    print(responses)

    return responses

def chunked_iterable(iterable, size):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, size))
        if not chunk:
            return
        yield chunk

def generate_responses(gen_model, gen_tokenizer, prompts, batch_size = 32) : 
    batch_responses = []
    
    for batch_index, batch_prompts in enumerate(chunked_iterable(prompts, batch_size)):
        print(f"[k={k} | Batch {batch_index}] Started generation...")
        start_time = time.time()
        
        batch_responses.extend(process_batch_prompts(batch_prompts, gen_tokenizer, gen_model, DEVICE))
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[k={k} | Batch {batch_index}] Time for generation = {elapsed_time:.2f} seconds")
    return batch_responses

In [ ]:
k_values = [3]
generated_responses = {}
for k in k_values:
    generated_responses[k] = generate_responses(gen_model, gen_tokenizer, prompts_k[k])
    print(f"Done with k = {k}!")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[k=1 | Batch 0] Started generation...
['<s>[INST] Give a concise answer to the question.\n Polymorphism in the CYSLTR2 gene has been associated with the inheritance of asthma in separate populations. The M201V variant of CYSLTR2 exhibits decreased responsiveness to LTD4, suggesting that this hypo-responsiveness underlies its asthma transmission-protecting effect.\nWhat is the effect of the M201V variant of CYSLTR2 on responsiveness to LTD4? [/INST]', '<s>[INST] Give a concise answer to the question.\n  Carbapenems are structurally similar to penicillins, but with a carbon atom replacing the sulfur atom in position 1. They are biosynthesized through a series of steps involving the condensation of malonyl-CoA with glutamate-5-semialdehyde, followed by the formation of the β-lactam and saturated carbapenam core.\nWhat is the key difference in the structure of carbapenems compared to penicillins? [/INST]', '<s>[INST] Give a concise answer to the question.\n  The 8-demicubic honeycomb is a 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['The M201V variant of CYSLTR2 exhibits decreased responsiveness to LTD4.', 'The key difference in the structure of carbapenems compared to penicillins is the replacement of the sulfur atom in position 1 with a carbon atom.', 'The 8-demicubic honeycomb has two types of facets: 8-demicubes and 8-orthoplex facets.', '5F-AMB is a controlled substance in the United States.', 'E2MATE is converted into estrone sulfamate by 17β-hydroxysteroid dehydrogenase.', '3-HO-PCP has high affinity for the NMDA receptor and opioid receptors.', 'The PAI-1 gene is located on chromosome 7.', '5F-AMB is a controlled substance, specifically a controlled research chemical.', '3-MeO-PCP binds to the NMDA receptor with higher affinity than PCP.', 'BARF is a non-coordinating anion.', 'The approximation for the electron heat capacity is proportional to the temperature and the density of states at the Fermi level.', 'The primary function of HLA-DP in the immune system is to serve as a cell surface receptor for fore

KeyboardInterrupt: 

### Evaluation

In [ ]:
def evaluate_bleu(df_true: pd.DataFrame, df_pred: pd.DataFrame, bleu_type: int):
    df_merged = pd.merge(df_true[['id', 'answer']], df_pred[['id', 'answer']], on='id', suffixes=('_true', '_pred'))

    weights = (1.0, 0.0, 0.0, 0.0) if bleu_type == 1 else (0.5, 0.5, 0.0, 0.0)
    
    bleu_scores = []

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for _, row in df_merged.iterrows():
            reference = row['answer_true'].split()
            candidate = row['answer_pred'].split() 

            bleu_score = sentence_bleu([reference], candidate, weights=weights)
            bleu_scores.append(bleu_score)
        
    mean_bleu = sum(bleu_scores) / len(bleu_scores)
    return mean_bleu

In [ ]:
df_true_faiss = questions_val[['id', 'answer']].copy()
df_pred_faiss = questions_val[['id']].copy()
df_pred_faiss['answer'] = generated_responses[k]

bleu_1 = evaluate_bleu(df_true_faiss, df_pred_faiss, 1)
bleu_2 = evaluate_bleu(df_true_faiss, df_pred_faiss, 2)

print(f"Le score BLEU-1 pour k = {k} est {bleu_1:.3f}.")
print(f"Le score BLEU-2 pour k = {k} est {bleu_2:.3f}. \n")

In [ ]:
df_export = questions_val[['id', 'question', 'answer']].copy()
df_export['generated_answer'] = generated_responses[k]
df_export.to_csv(f"generated_responses.csv", index=False, encoding='utf-8')